In [1]:
# -*- coding: utf-8
# Abraji (https://www.abraji.org.br)
# Reinaldo Chaves (reinaldo@abraji.org.br)
# Acessa as respostas aos recursos administrativos em pedidos da LAI no Estado de São Paulo
# Faz: o download dos PDFs, 
# transforma os arquivos em imagem, 
# faz uma leitura OCR, 
# captura o texto 
# e filtra por resposta deferidas e indeferidas
# Cria dataframe e arquivos com os resultados
# A rotina de leitura OCR foi adaptada de:
# https://www.geeksforgeeks.org/python-reading-contents-of-pdf-using-ocr-optical-character-recognition/
#

In [2]:
import requests
from urllib.request import urlopen
import wget
from bs4 import BeautifulSoup
import pandas as pd
import os
import time

In [3]:
from PIL import Image 
import pytesseract 
import sys 
from pdf2image import convert_from_path 

In [3]:
def captura_texto(arquivo):
    PDF_file = arquivo

    # Armazena todas as páginas do PDF em uma variável
    pages = convert_from_path(PDF_file, 500) 

    # Contador para armazenar imagens de cada página do PDF na imagem
    image_counter = 1

    # Itera todas as páginas armazenadas acima
    for page in pages: 
        # Declarando o nome do arquivo para cada página do PDF como JPG
        # Para cada página, o nome do arquivo será:
        # PDF page 1 -> page_1.jpg 
        # PDF page 2 -> page_2.jpg 
        # PDF page 3 -> page_3.jpg 
        # .... 
        # PDF page n -> page_n.jpg 
        filename = "page_"+str(image_counter)+".jpg"
      
        # Salve a imagem da página no sistema
        page.save(filename, 'JPEG') 
  
        # Incremente o contador para atualizar o nome do arquivo
        image_counter = image_counter + 1

    ''' 
    Parte 2 - Reconhecendo o texto das imagens usando o OCR
    '''
    
    # Variável para obter a contagem do número total de páginas
    filelimit = image_counter-1
  
    # Criando um arquivo de texto para gravar a saída
    outfile = "out_text.txt"
  
    # Abra o arquivo no modo de acréscimo para que
    # todo o conteúdo de todas as imagens seja adicionado ao mesmo arquivo
    f = open(outfile, "a") 
  
    # Iterar de 1 para o número total de páginas

    for i in range(1, filelimit + 1): 
        # Defina o nome do arquivo para reconhecer o texto
        # Novamente, esses arquivos serão:
        # page_1.jpg 
        # page_2.jpg 
        # .... 
        # page_n.jpg 
        filename = "page_"+str(i)+".jpg"
          
        # Reconhecer o texto como string na imagem usando pytesserct
        text = str(((pytesseract.image_to_string(Image.open(filename))))) 
  
        # O texto reconhecido é armazenado em uma variável de texto
        # Qualquer processamento de string pode ser aplicado ao texto
        # Aqui, a formatação básica foi feita:
        # Em muitos PDFs, no final da linha, se uma palavra não puder
        # ser totalmente escrita, um 'hífen' é adicionado.
        # O restante da palavra está escrito na próxima linha
        # Por exemplo: Este é um exemplo de texto desta palavra aqui GeeksF-
        # orGeeks está na metade da primeira linha, permanecendo na próxima.
        # Para remover isso, substituímos todos os '-\n' por ''.  
        text = text.replace('-\n', '')     
  
        # Por fim, escreve o texto processado no arquivo.
        f.write(text) 
  
    # Feche o arquivo depois de escrever todo o texto
    f.close() 
    
    # Abre o arquivo criado
    f=open("out_text.txt", "r")
    
    if f.mode == 'r':
        # Armazena o conteudo
        contents =f.read()
        conteudo = contents
    else:
        conteudo = ""
    
    # Retorna o conteudo para iteração principal
    return conteudo

In [ ]:
# URL onde estão os links dos PDFs

In [4]:
pagina = urlopen("http://www.ouvidoriageral.sp.gov.br/decisoesLAI.html")

In [5]:
# Procura o local com os endereços dos arquivos

In [6]:
sopa =  BeautifulSoup(pagina, "lxml")

In [13]:
pdfs = sopa.findAll(id='dataTable')

In [14]:
pdfs

[<table cellspacing="0" class="table table-bordered" id="dataTable" width="100%">
 <thead class="thead-dark">
 <tr>
 <th width="3%">Ano</th>
 <th width="2%">Nº</th>
 <th width="30%">Órgão</th>
 <th width="45%">Resumo</th>
 <th width="15%">Decisão</th>
 <th width="5%">Ver</th>
 </tr>
 </thead>
 <tfoot class="thead-dark">
 <tr>
 <th>Ano</th>
 <th>Nº</th>
 <th>Órgão</th>
 <th>Resumo</th>
 <th>Decisão</th>
 <th>Ver</th>
 </tr>
 </tfoot>
 <tbody>
 <!--2020-->
 <tr>
 <td align="center">2020</td>
 <td align="right">1</td>
 <td>Secretaria    Estadual da Segurança Pública - SSP</td>
 <td>Acesso a    informações sobre decisões proferidas anteriormente. Informações incompletas.    Envio extemporâneo</td>
 <td>Perda de objeto</td>
 <td align="center"><a href="LAI/arquivos/2020/Decisão 2ª Instância 001-2020.pdf" target="_blank"><i class="fas fa-2x fa-file-pdf"></i></a></td>
 </tr>
 <tr>
 <td align="center">2020</td>
 <td align="right">2</td>
 <td>Secretaria    Estadual da Saúde</td>
 <td>Acesso a i

In [74]:
lista = []
fixo = "http://www.ouvidoriageral.sp.gov.br/"

In [75]:
# Faz um iteração na tabela de links e armazena os endereços

In [76]:
for table_ele in pdfs:
    
    for row in table_ele.findAll('tr'):
        
        cols = row.findAll('td')
       
        if not cols:
            None
        else:
            resumo = cols[-3].text
            decisao = cols[-2].text
            orgao = cols[-4].text
            numero = cols[-5].text
            ano = cols[-6].text
            url = cols[-1]
            exato = url.find('a').get('href')
            site = fixo + exato
            
            dicionario = {"site": str(site).strip(),
                          "numero": str(numero).strip(),
                          "ano": str(ano).strip(),
                          "resumo": str(resumo).strip(),
                          "decisao": str(decisao).strip(),
                          "orgao": str(orgao).strip()
                             }
            
            lista.append(dicionario)

In [77]:
# Cria um dataframe

In [78]:
df_pdfs = pd.DataFrame(lista)

In [79]:
df_pdfs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1548 entries, 0 to 1547
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   site     1548 non-null   object
 1   numero   1548 non-null   object
 2   ano      1548 non-null   object
 3   resumo   1548 non-null   object
 4   decisao  1548 non-null   object
 5   orgao    1548 non-null   object
dtypes: object(6)
memory usage: 72.7+ KB


In [80]:
df_pdfs.reset_index().tail()

,index,site,numero,ano,resumo,decisao,orgao
1543,1543,http://www.ouvidoriageral.sp.gov.br/LAI/arquiv...,365,2016,Vagas para cargo de Professor de Lingua Esp...,Negado provimento,Secretaria Estadual da Educação - SEE
1544,1544,http://www.ouvidoriageral.sp.gov.br/LAI/arquiv...,366,2016,Iimóveis alienados pela Secretaria Estadual...,Provimento,Secretaria de Logistica e Transportes -Depa...
1545,1545,http://www.ouvidoriageral.sp.gov.br/LAI/arquiv...,367,2016,Não Localizada,,Não Localizada
1546,1546,http://www.ouvidoriageral.sp.gov.br/#,368,2016,"Processo disciplinar, laudo periciais",Negado provimento,Secretaria Estadual da Segurança Pública - ...
1547,1547,http://www.ouvidoriageral.sp.gov.br/LAI/arquiv...,369,2016,Contratos para realização de show,Provimento,Secretaria Estadual da Cultura


In [84]:
df_pdfs.to_excel('lista_da_tabela_ouvidoria_sp.xlsx',sheet_name='Sheet1', index=False)

In [15]:
# Seta o diretório de trabalho 

In [23]:
print (os.getcwd())

/home/abraji/Documentos/Code/lai_sp/repo


In [24]:
os.chdir("/home/abraji/Documentos/Code/lai_sp/repo/pdfs")

In [25]:
print (os.getcwd())

/home/abraji/Documentos/Code/lai_sp/repo/pdfs


In [26]:
dirname = "/home/abraji/Documentos/Code/lai_sp/repo/pdfs"

In [ ]:
# Cria uma lista vazia para os resultados

In [16]:
lista_final = []

In [ ]:
# Inicia iteração no dataframe de links de PDFs

In [ ]:
for num, row in df_pdfs.iterrows():
    # Cria variáveis com os itens do PDFs
    link = row['site']
    numero = row['numero']
    ano = row['ano']
    # O nome do arquivo fica nessa posição, mas pode ter um caracter indevido
    arquivo = link[50:]
    arquivo = arquivo.replace("/", "")
    
    print(arquivo)
    print(link)
    #time.sleep(2)
    
    try:
        # Faz o download do PDF
        wget.download(link)
        
        # Faz a rotina OCR, baseado no nome do arquivo na função acima
        texto = captura_texto(arquivo)
        
        # Cria um dicionário com dados do PDFs e seu conteúdo retornado
        dicionario = {"site": str(link).strip(),
                      "numero": str(numero).strip(),
                      "ano": str(ano).strip(),
                      "texto_do_documento": str(texto).strip()
                             }
        # Armazena na lista criada
        lista_final.append(dicionario)
        
        # Apaga o arquivo de conteudo do PDF atual
        os.remove("out_text.txt")
        # Apaga as imagens OCR do PDF atual
        test = os.listdir(dirname)
        for item in test:
            if item.endswith(".jpg"):
                os.remove(os.path.join(dirname, item))
        
    except:
        print("ERRRO")
        pass

In [ ]:
# Cria dataframe a partir da lista de conteudos de PDFs

In [18]:
df_pdfs_final = pd.DataFrame(lista_final)

In [19]:
df_pdfs_final.reset_index().head()

,index,site,numero,ano,texto_do_documento
0,0,http://www.ouvidoriageral.sp.gov.br/decisoes/2...,004,2019,GOVERNO DO ESTADO DE SAO PAULO\nOUVIDORIA GERA...
1,1,http://www.ouvidoriageral.sp.gov.br/decisoes/2...,005,2019,GOVERNO DO ESTADO DE SAO PAULO\nOUVIDORIA GERA...
2,2,http://www.ouvidoriageral.sp.gov.br/decisoes/2...,006,2019,—\n\nGOVERNO DO ESTADO DE SAO PAULO\nOUVIDORIA...
3,3,http://www.ouvidoriageral.sp.gov.br/decisoes/2...,007,2019,GOVERNO DO ESTADO DE SAO PAULO\nOUVIDORIA GERA...
4,4,http://www.ouvidoriageral.sp.gov.br/decisoes/2...,008,2019,GOVERNO DO ESTADO DE SAO PAULO\nOUVIDORIA GERA...


In [27]:
df_pdfs_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1253 entries, 0 to 1252
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   site                1253 non-null   object
 1   numero              1253 non-null   object
 2   ano                 1253 non-null   object
 3   texto_do_documento  1253 non-null   object
 4   texto_minusculo     1253 non-null   object
dtypes: object(5)
memory usage: 49.1+ KB


In [21]:
df_pdfs_final.to_csv("../laisp_textos.csv", index=False)

In [ ]:
# Carrega arquivo salvo se não quiser repetir o download OCR

In [59]:
kwargs = {'sep': ',', 'dtype': str, 'encoding': 'utf-8'}
df_pdfs_final = pd.read_csv("laisp_textos.csv", **kwargs)

In [19]:
teste = df_pdfs_final[pd.isnull(df_pdfs_final['texto_do_documento'])]

In [20]:
teste

,site,numero,ano,texto_do_documento
463,http://www.ouvidoriageral.sp.gov.br/decisoes/2...,122,2018,NaN


In [21]:
teste.to_csv("teste.csv", index=False)

In [60]:
df_pdfs_final = df_pdfs_final[pd.notnull(df_pdfs_final['texto_do_documento'])]

In [61]:
df_pdfs_final.shape

(1252, 4)

In [62]:
# Cria um campo em separado copia para fazer buscas

In [63]:
df_pdfs_final['texto_minusculo'] = df_pdfs_final['texto_do_documento']
df_pdfs_final['texto_minusculo'] = df_pdfs_final['texto_minusculo'].str.lower()

In [64]:
# Procura termos relacionados a indeferimento

In [65]:
#search_list = ["indeferido", "indeferimento", "negado provimento", "recurso nao conhecido", "recurso não conhecido", "negar-lhe provimento", "nego-lhe provimento", "negando-lhe provimento", "nego provimento"]
search_list = ["negar-lhe provimento", "nego-lhe provimento", "negando-lhe provimento", "nego provimento", "nego seu provimento"]
mask = df_pdfs_final['texto_minusculo'].str.contains('|'.join(search_list))
indeferido = df_pdfs_final[mask]
indeferido.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 301 entries, 0 to 1251
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   site                301 non-null    object
 1   numero              301 non-null    object
 2   ano                 301 non-null    object
 3   texto_do_documento  301 non-null    object
 4   texto_minusculo     301 non-null    object
dtypes: object(5)
memory usage: 14.1+ KB


In [66]:
indeferido.to_csv("indeferidos.csv", index=False)

In [67]:
# Procura termos relacionados a deferimento

In [68]:
#search_list = ["deferido", "deferimento", "provimento recursal", "dando-lhe provimento", "dando-lhe parcial provimento", "dou-lhe provimento", "dou-lhe parcial provimento", "dou provimento", "dou parcial provimento"]
search_list = ["dando-lhe provimento", "dando-lhe parcial provimento", "dou-lhe provimento", "dou-lhe parcial provimento", "dou provimento", "dou parcial provimento"]
mask = df_pdfs_final['texto_minusculo'].str.contains('|'.join(search_list))
deferido = df_pdfs_final[mask]
deferido.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 239 entries, 5 to 1252
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   site                239 non-null    object
 1   numero              239 non-null    object
 2   ano                 239 non-null    object
 3   texto_do_documento  239 non-null    object
 4   texto_minusculo     239 non-null    object
dtypes: object(5)
memory usage: 11.2+ KB


In [69]:
deferido.to_csv("deferidos.csv", index=False)

In [35]:
# Procura neutro

In [72]:
#search_list = ["perda de objeto", "perda superveniente de objeto"]
search_list = ["perda superveniente de objeto"]
mask = df_pdfs_final['texto_minusculo'].str.contains('|'.join(search_list))
neutro = df_pdfs_final[mask]
neutro.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 268 entries, 3 to 1248
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   site                268 non-null    object
 1   numero              268 non-null    object
 2   ano                 268 non-null    object
 3   texto_do_documento  268 non-null    object
 4   texto_minusculo     268 non-null    object
dtypes: object(5)
memory usage: 12.6+ KB


In [73]:
neutro.to_csv("neutros.csv", index=False)

In [ ]:
# Não é recurso válido

In [74]:
search_list = ["não conheço do recurso", "nao conheco do recurso"]
mask = df_pdfs_final['texto_minusculo'].str.contains('|'.join(search_list))
naovalido = df_pdfs_final[mask]
naovalido.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7 entries, 41 to 1020
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   site                7 non-null      object
 1   numero              7 non-null      object
 2   ano                 7 non-null      object
 3   texto_do_documento  7 non-null      object
 4   texto_minusculo     7 non-null      object
dtypes: object(5)
memory usage: 336.0+ bytes


In [75]:
naovalido.to_csv("recurso_nao_valido.csv", index=False)

In [76]:
# Nenhma dessas

In [77]:
#search_list = ["indeferido", "indeferimento", "negado provimento", "recurso nao conhecido", "recurso não conhecido", "negar-lhe provimento", "nego-lhe provimento", "negando-lhe provimento", "nego provimento", "deferido", "deferimento", "provimento recursal", "dando-lhe provimento", "dando-lhe parcial provimento", "dou-lhe provimento", "dou-lhe parcial provimento", "dou provimento", "dou parcial provimento", "perda de objeto", "perda superveniente de objeto"]
search_list = ["negar-lhe provimento", "nego-lhe provimento", "negando-lhe provimento", "nego provimento", "nego seu provimento", "dando-lhe provimento", "dando-lhe parcial provimento", "dou-lhe provimento", "dou-lhe parcial provimento", "dou provimento", "dou parcial provimento",  "perda superveniente de objeto", "não conheço do recurso", "nao conheco do recurso"]
mask = ~df_pdfs_final['texto_minusculo'].str.contains('|'.join(search_list))
defora = df_pdfs_final[mask]
defora.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 439 entries, 1 to 1246
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   site                439 non-null    object
 1   numero              439 non-null    object
 2   ano                 439 non-null    object
 3   texto_do_documento  439 non-null    object
 4   texto_minusculo     439 non-null    object
dtypes: object(5)
memory usage: 20.6+ KB


In [78]:
defora.to_csv("nao_encontrado.csv", index=False)